In [120]:
import os
import pandas as pd
import numpy as np



# Load dataset

In [121]:
path = '../data/nlp/20news-bydate-train/'
array = []
for dir1 in os.listdir(path):
    if os.path.isdir(os.path.join(path, dir1)):
        
        for file1 in os.listdir(os.path.join(path, dir1)):
            #print(os.path.join(path, dir1, file1))
            dct = {'From': '', 'Lines': '', 'Subject': '', 'Organization': ''}
            with open(os.path.join(path, dir1, file1), 'r', encoding='utf-8') as f:
                try:
                    lines = f.readlines()
                except:
                    print('Decoding error: {0}'.format(os.path.join(path, dir1, file1)))
            for i, line in enumerate(lines):
                splitted = line.strip().split(': ')
                if len(splitted) < 2:
                    break
                dct[splitted[0]] = splitted[1]

            message = '\n'.join(lines[i:])
            array.append((dct['From'], dct['Lines'], dct['Subject'], dct['Organization'], message, dir1))


Decoding error: ../data/nlp/20news-bydate-train/misc.forsale/76548
Decoding error: ../data/nlp/20news-bydate-train/misc.forsale/76112
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51594
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/50467
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51592
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51593
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51916
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51865
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51941
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51892
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51904
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51591
Decoding error: ../data/nlp/20news-bydate-train/comp.sys.mac.hardware/51917
Decoding error: ../data/nlp/20

In [122]:
df = pd.DataFrame(array, columns=['From', 'Lines', 'Subject', 'Organization', 'Message', 'Category'])

In [123]:
path = '../data/nlp/20news-bydate-test/'

array = []
for dir1 in os.listdir(path):
    if os.path.isdir(os.path.join(path, dir1)):
        
        for file1 in os.listdir(os.path.join(path, dir1)):
            #print(os.path.join(path, dir1, file1))
            dct = {'From': '', 'Lines': '', 'Subject': '', 'Organization': ''}
            with open(os.path.join(path, dir1, file1), 'r', encoding='utf-8') as f:
                try:
                    lines = f.readlines()
                except:
                    print('Decoding error: {0}'.format(os.path.join(path, dir1, file1)))
            for i, line in enumerate(lines):
                splitted = line.strip().split(': ')
                if len(splitted) < 2:
                    break
                dct[splitted[0]] = splitted[1]

            message = '\n'.join(lines[i:])
            array.append((dct['From'], dct['Lines'], dct['Subject'], dct['Organization'], message, dir1))

df_test = pd.DataFrame(array, columns=['From', 'Lines', 'Subject', 'Organization', 'Message', 'Category'])

Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54000
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/53879
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54036
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54769
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54042
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.hockey/54516
Decoding error: ../data/nlp/20news-bydate-test/misc.forsale/76361
Decoding error: ../data/nlp/20news-bydate-test/misc.forsale/76843
Decoding error: ../data/nlp/20news-bydate-test/comp.sys.mac.hardware/52196
Decoding error: ../data/nlp/20news-bydate-test/comp.sys.mac.hardware/52033
Decoding error: ../data/nlp/20news-bydate-test/comp.sys.mac.hardware/52165
Decoding error: ../data/nlp/20news-bydate-test/comp.sys.mac.hardware/52164
Decoding error: ../data/nlp/20news-bydate-test/comp.windows.x/68001
Decoding error: ../data/nlp/20news-bydate-test/rec.sport.baseball/104984
Decodin

In [124]:
df_train = df
print(df_train.shape)
print(df_test.shape)

(11314, 6)
(7532, 6)


In [125]:
from sklearn.preprocessing import LabelEncoder
df_train['Target'] = LabelEncoder().fit_transform(df_train['Category'])
df_test['Target'] = LabelEncoder().fit_transform(df_test['Category'])

In [126]:
df_train['Re'] = (df_train['Subject'] == 'Re').map(lambda x: 1 if x else 0)
df_test['Re'] = (df_test['Subject'] == 'Re').map(lambda x: 1 if x else 0)
df_train['Subject'] = df_train['Subject'].map(lambda x: '' if x == 'Re' else x)
df_test['Subject'] = df_test['Subject'].map(lambda x: '' if x == 'Re' else x)

# Get features

In [127]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import re

stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if \
              len([letter for letter in word if letter.isalpha()]) > 2 and '.' not in word]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


In [128]:
tfidf_vectorizer_message = TfidfVectorizer(max_df=0.5, max_features=200000,
                                 min_df=10, stop_words='english',
                                use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_message_train = tfidf_vectorizer_message.fit_transform(df_train['Message'])

In [129]:
tfidf_message_test = tfidf_vectorizer_message.transform(df_test['Message'])

In [130]:
tfidf_vectorizer_subject = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=2, stop_words='english',
                                use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_subject_train = tfidf_vectorizer_subject.fit_transform(df_train['Subject'])
tfidf_subject_test = tfidf_vectorizer_subject.transform(df_test['Subject'])

# Classify

## Train only with messages

In [131]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(tfidf_message_train, df_train['Target'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [132]:
print('Accuracy on the test set, using only messages: {0}'\
      .format(clf.score(tfidf_message_test, df_test['Target'])))

Accuracy on the test set, using only messages: 0.7693839617631439


## Train also with subjects

In [133]:
from scipy.sparse import hstack
train_matrix = hstack([tfidf_message_train, tfidf_subject_train])
test_matrix = hstack([tfidf_message_test, tfidf_subject_test])

clf_subject = MultinomialNB()
clf_subject.fit(train_matrix, df_train['Target'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [134]:
print('Accuracy on the test set, using messages and subjects: {0}'\
      .format(clf_subject.score(test_matrix, df_test['Target'])))

Accuracy on the test set, using messages and subjects: 0.7749601699415826


# Cluster

In [135]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=20, random_state=0)
km.fit_predict(train_matrix)

array([18,  5, 18, ...,  5,  2,  6], dtype=int32)

In [136]:
from sklearn import metrics

labels = df_train['Target']
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(train_matrix, km.labels_, sample_size=1000))

Homogeneity: 0.376
Completeness: 0.450
V-measure: 0.410
Adjusted Rand-Index: 0.134
Silhouette Coefficient: -0.039


In [140]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms1 = tfidf_vectorizer_message.get_feature_names()
terms2 = tfidf_vectorizer_subject.get_feature_names()
terms = terms1 + terms2
for i in range(20):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: drive forsal scsi disk drive ide hard drive hard floppi control
Cluster 1: god christian jesus believ bibl peopl christ say faith church
Cluster 2: use sale simm work mac appl chip comput like doe
Cluster 3: car bike ride articl engin car just like drive look
Cluster 4: armenian clinton israel isra turkish arab armenian turk armenia turkey
Cluster 5: card problem card driver video video monitor monitor video card window
Cluster 6: articl just like think ani say use know post time
Cluster 7: want thank look pleas email e-mail info want game anyon ani
Cluster 8: sale sale offer condit best offer sell make offer ask new excel
Cluster 9: file file window format directori program use imag format convert
Cluster 10: geb gordon bank shame surrend soon surrend soon shame surrend geb shame surrend geb shame chastiti intellect geb intellect geb intellect geb shame
Cluster 11: question question thank ani doe answer use just know x-termin
Cluster 12: space space orbit henri launch msg m

In [138]:
len(terms)

35182

In [139]:
order_centroids.shape

(20, 20081)

## Only with messages

In [141]:
km = KMeans(n_clusters=20, random_state=0)
km.fit_predict(tfidf_message_train)

array([17,  3, 18, ...,  3,  7, 18], dtype=int32)

In [142]:

labels = df_train['Target']
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(tfidf_message_train, km.labels_, sample_size=1000))

Homogeneity: 0.388
Completeness: 0.478
V-measure: 0.428
Adjusted Rand-Index: 0.122
Silhouette Coefficient: -0.043


In [143]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer_message.get_feature_names()
for i in range(20):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: thank anyon ani doe anyon pleas know doe anyon know thank advanc advanc
Cluster 1: space henri orbit launch moon nasa lunar satellit shuttl earth
Cluster 2: file directori program window use imag ftp format convert ani
Cluster 3: card driver video video card window bus vesa ati use diamond
Cluster 4: car bike ride articl just dod like engin ani good
Cluster 5: drive scsi disk ide hard drive hard floppi control boot problem
Cluster 6: christian jesus church god bibl religion peopl christ believ faith
Cluster 7: use simm sale offer mac price appl printer port monitor
Cluster 8: georgia univers georgia michael covington covington mcovingt athen georgia athen michael athen georgia amateur amateur radio n4tmi
Cluster 9: key encrypt chip clipper secur escrow govern use phone nsa
Cluster 10: israel isra arab jew palestinian lebanes kill lebanon polici soldier
Cluster 11: game team player play year win season hockey score fan
Cluster 12: armenian turkish turk armenia turkey greek az